# Lab 2 - Automated Data Quality Assurance Pipeline

## Temperature, MN Land Cover Data, Minnesota Elevation

### import packages

In [24]:
import arcpy
import requests
import zipfile
import os
import json
import pandas as pd
import psycopg2
from psycopg2 import sql
from io import StringIO
import arcgis
from arcgis.features import GeoAccessor, GeoSeriesAccessor
import os

### Function for download data

In [28]:
def data_download(data_url, target_folder_name, file_type='zip'):
    # Download the data
    response = requests.get(data_url)
    
    if file_type == 'zip':
        # Extract filename from the URL
        filename = os.path.basename(data_url)

        # Save the data to file
        with open(filename, 'wb') as file:
            file.write(response.content)
        
    #if file_type == 'zip':
        # Extract the contents of the zip file
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            # Construct the full path for the target folder
            target_folder = os.path.join(r'C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Lab 2', target_folder_name)

            # Check if the directory exists, if not, create it
            if not os.path.exists(target_folder):
                os.makedirs(target_folder)

            # Extract the contents of the zip file to the target directory
            zip_ref.extractall(target_folder)
    elif file_type == 'geojson':
        
         # Extract filename from the URL
        filename = target_folder_name

        # Save the data to file
        with open(filename, 'wb') as file:
            file.write(response.content)
            
        # Construct the full path for the target folder
        target_folder = os.path.join(r'C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Lab 2', target_folder_name)
        
        # Check if the directory exists, if not, create it
        if not os.path.exists(target_folder):
            os.makedirs(target_folder)
        
        # Save the data to file
        with open(os.path.join(target_folder, filename), 'wb') as file:
            file.write(response.content)


In [30]:
import requests
import os
import zipfile

def data_download(data_url, target_folder_name, file_type='zip'):
    response = requests.get(data_url)
    if response.status_code == 200:  # Check if the download was successful
        target_folder = os.path.join(r'C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Lab 2', target_folder_name)
        if not os.path.exists(target_folder):
            os.makedirs(target_folder)

        filename = os.path.basename(data_url)  # Extract filename from the URL

        if file_type == 'zip':
            # Save the ZIP file in the target folder
            zip_file_path = os.path.join(target_folder, filename)
            with open(zip_file_path, 'wb') as file:
                file.write(response.content)

            # Extract the ZIP file
            with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
                zip_ref.extractall(target_folder)

        elif file_type == 'geojson':
            # Define a proper filename for the GeoJSON file, ensuring it ends with '.geojson'
            geojson_file_path = os.path.join(target_folder, target_folder_name + '.geojson')
            with open(geojson_file_path, 'wb') as file:
                file.write(response.content)
    else:
        print(f"Failed to download data. Status code: {response.status_code}")


In [31]:
# Print the current working directory
print("Current working directory: {0}".format(os.getcwd()))

Current working directory: C:\Windows\System32


### MN boundary (study area)
For Minnesota boundary, I created the boundary box and also download the data from Minnesota Geospatial Commons.

In [20]:
# define Minnesota boundary box
north = 49.5
east = -89.0
south = 43.0
west = -97.5

In [33]:
data_download(
    'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/bdry_state_of_minnesota/fgdb_bdry_state_of_minnesota.zip',
    'mn_boundary')


### Temperature
for temperature, make sure the data is insude the MN boundary box and filter the temperature between degree -10 and 50

In [34]:
# download data
data_download('https://mesonet.agron.iastate.edu/api/1/daily.geojson?network=MN_RWIS&month=3&year=2023',
              'mn_temperature', 'geojson')

In [44]:
# filter the data
with open(r'C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Lab 2\mn_temperature\mn_temperature.geojson', 'r') as f:
    temperature_data = json.load(f)


# delete unnecessary items
clean_tem_data = []
for feature in temperature_data['features']:
    tem_feature = {
        "station": feature['properties']['station'],
        "date": feature['properties']['date'],
        "max_tmpf": feature['properties']['max_tmpf'],
        "min_tmpf": feature['properties']['min_tmpf'],
        "name": feature['properties']['name'],
        "geometry": feature['geometry']
    }
    clean_tem_data.append(tem_feature)

# filter data
filtered_tem_data = []
rejected_tem_data = []

for feature in clean_tem_data:
    
    coordinate_x = feature['geometry']['coordinates'][0]
    coordinate_y = feature['geometry']['coordinates'][1]
    max_tmp = feature['max_tmpf']
    min_tmp = feature['min_tmpf']
    
    # Check if max_tmp and min_tmp are not null
    if max_tmp is not None and min_tmp is not None:
        # drop the data out of the MN boundary box and not in temperature range
        if (-97.5 <= coordinate_x <= -89.0 and 43.0 <= coordinate_y <= 49.5) and (50.0 >= max_tmp >= -10.0 and -10.0 <= min_tmp <= 50.0):
            filtered_tem_data.append(feature)
        else:
            rejected_tem_data.append(feature)
    else:
        rejected_tem_data.append(feature)

# check how many data in the study area
print("Number of features within MN boundary box and in temperature range:", len(filtered_tem_data))

# print the filtered data
filtered_df = pd.DataFrame(filtered_tem_data)

filtered_df

Number of features within MN boundary box and in temperature range: 4637


,station,date,max_tmpf,min_tmpf,name,geometry
0,MN001,2023-03-01,38.840023,26.780000,Twin Lakes I-35 Mile Post 1,"{'type': 'Point', 'coordinates': [-93.35405731..."
1,MN002,2023-03-01,33.980000,18.680000,Silver Lake TH 7 Mile Post 1,"{'type': 'Point', 'coordinates': [-94.1191, 44..."
2,MN003,2023-03-01,37.399990,23.360010,Little Chicago I-35 Mile Post 70,"{'type': 'Point', 'coordinates': [-93.29242706..."
3,MN004,2023-03-01,30.019978,16.519978,Rush City I-35 Mile Post 157,"{'type': 'Point', 'coordinates': [-92.99275207..."
4,MN005,2023-03-01,29.119978,15.980000,Rutledge I-35 Mile Post 198,"{'type': 'Point', 'coordinates': [-92.83856201..."
...,...,...,...,...,...,...
4632,MN158,2023-03-31,33.080000,28.219978,U.S.75 - Canby - MP 84.0 MN US MNDOT,"{'type': 'Point', 'coordinates': [-96.27693176..."
4633,MN159,2023-03-31,33.080000,27.680000,U.S.12 - Atwater - MP 85.4 MN US MNDOT,"{'type': 'Point', 'coordinates': [-94.81175994..."
4634,MN160,2023-03-31,31.819979,28.399988,U.S.14 - Florence - MP 21.2 MN US MNDOT,"{'type': 'Point', 'coordinates': [-96.04634094..."
4635,MN161,2023-03-31,35.419980,30.919977,U.S.12 - Delano - MP 140.4 MN US MNDOT,"{'type': 'Point', 'coordinates': [-93.76606750..."


In [45]:
# print the rejected data
print("Number of rejected features:", len(rejected_tem_data))

rejected_df = pd.DataFrame(rejected_tem_data)

rejected_df

Number of rejected features: 106


,station,date,max_tmpf,min_tmpf,name,geometry
0,MN006,2023-03-01,NaN,NaN,Beaver Creek I-90 Mile Post 4,"{'type': 'Point', 'coordinates': [-96.3779, 43..."
1,MN006,2023-03-02,NaN,NaN,Beaver Creek I-90 Mile Post 4,"{'type': 'Point', 'coordinates': [-96.3779, 43..."
2,MN013,2023-03-02,20.480000,-12.460006,Dilworth I-94 Mile Post 5,"{'type': 'Point', 'coordinates': [-96.66813659..."
3,MN018,2023-03-02,25.880000,-13.900011,Shooks MN-1 Mile Post 34,"{'type': 'Point', 'coordinates': [-94.43242645..."
4,MN019,2023-03-02,26.240023,-14.620000,Ely MN-1 Mile Post 275,"{'type': 'Point', 'coordinates': [-92.04811096..."
...,...,...,...,...,...,...
101,MN114,2023-03-29,16.160011,-17.500011,T.H.9 - Beltrami - MP 212.5 MN US MNDOT,"{'type': 'Point', 'coordinates': [-96.53217315..."
102,MN118,2023-03-29,17.419977,-12.639989,T.H.1 - Goodridge - MP 71.8 MN US MNDOT,"{'type': 'Point', 'coordinates': [-95.62496185..."
103,MN135,2023-03-29,16.519978,-11.020000,U.S.75 - Kent - MP 222 MN US MNDOT,"{'type': 'Point', 'coordinates': [-96.71227264..."
104,MN001,2023-03-31,50.899990,32.899990,Twin Lakes I-35 Mile Post 1,"{'type': 'Point', 'coordinates': [-93.35405731..."


In [47]:
# store the clean data 
with open(r'C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Lab 2\mn_temperature\filtered_tem_data.json', 'w') as f:
    json.dump(filtered_tem_data, f)


### MN Land Cover Data
for land cover, Here I make sure the data is insude the MN boundary box and filter the value != 0 (Unclassified)

In [48]:
# download data
data_download('https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/biota_landcover_nlcd_mn_2019/tif_biota_landcover_nlcd_mn_2019.zip', 
              'mn_landcover')

In [50]:
# clip the raster
mnlc = r'C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Lab 2\mn_landcover\NLCD_2019_Land_Cover.tif'

# Minnesota boundary gdb
mn_boundary_gdb = r'C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Lab 2\mn_boundary\bdry_state_of_minnesota.gdb\state_of_minnesota'

# output path
output_raster = r'C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Lab 2\clipped_mnlc.tif'

# use Minnesota boundary gdb to create extent
mn_boundary_feature = arcpy.management.CopyFeatures(mn_boundary_gdb, arcpy.Geometry())

# clip
arcpy.management.Clip(mnlc, mn_boundary_feature, output_raster)

print("Clipping completed. Clipped image saved as:", output_raster)

Clipping completed. Clipped image saved as: C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Lab 2\clipped_mnlc.tif


### Minnesota Elevation
for dem, data should insude the MN boundary box

In [1]:
# dowmload dem data
data_download('https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/elev_30m_digital_elevation_model/fgdb_elev_30m_digital_elevation_model.zip', 
              'mn_dem')

NameError: name 'data_download' is not defined

In [66]:
# clip the raster
dem = r'C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Lab 2\mn_dem\elev_30m_digital_elevation_model.gdb\digital_elevation_model_30m'

# Minnesota boundary gdb
mn_boundary_gdb = r'C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Lab 2\mn_boundary\bdry_state_of_minnesota.gdb\state_of_minnesota'

# output path
output_raster = r'C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Lab 2\Clipped_dem'

# use Minnesota boundary gdb to create extent
mn_boundary_feature = arcpy.management.CopyFeatures(mn_boundary_gdb, arcpy.Geometry())

# clip
arcpy.management.Clip(dem, mn_boundary_feature, output_raster)

#print("Clipping completed. Clipped raster saved as:", output_raster)



<Result 'C:\\Users\\samik\\OneDrive\\Documents\\ArcGIS\\Projects\\Lab 2\\Clipped_dem'>

#### Data source of your choosing needed for your project

In [18]:
## Evapotranspiration data
ndawn_url= r"https://ndawn.ndsu.nodak.edu/table.csv?station=78&station=111&station=98&station=162&station=174&station=142&station=164&station=138&station=161&station=9&station=160&station=159&station=10&station=118&station=56&station=165&station=11&station=12&station=58&station=13&station=84&station=55&station=179&station=7&station=186&station=87&station=14&station=15&station=96&station=191&station=16&station=201&station=137&station=124&station=143&station=17&station=85&station=140&station=134&station=18&station=136&station=65&station=104&station=99&station=192&station=19&station=129&station=20&station=101&station=166&station=178&station=81&station=21&station=97&station=22&station=75&station=184&station=2&station=172&station=139&station=158&station=23&station=157&station=62&station=86&station=24&station=89&station=126&station=167&station=93&station=183&station=90&station=25&station=83&station=107&station=156&station=77&station=26&station=155&station=70&station=127&station=144&station=27&station=173&station=132&station=28&station=195&station=185&station=29&station=30&station=154&station=31&station=187&station=102&station=32&station=119&station=4&station=80&station=33&station=59&station=153&station=105&station=82&station=34&station=198&station=72&station=135&station=35&station=76&station=120&station=141&station=109&station=36&station=79&station=193&station=71&station=37&station=38&station=189&station=39&station=130&station=73&station=188&station=40&station=41&station=54&station=69&station=194&station=145&station=113&station=128&station=42&station=43&station=103&station=171&station=116&station=196&station=88&station=114&station=3&station=163&station=200&station=64&station=115&station=168&station=67&station=175&station=146&station=170&station=197&station=44&station=133&station=106&station=100&station=121&station=45&station=46&station=61&station=66&station=181&station=74&station=60&station=199&station=125&station=176&station=177&station=8&station=180&station=204&station=47&station=122&station=108&station=5&station=152&station=48&station=151&station=147&station=68&station=169&station=49&station=50&station=91&station=182&station=117&station=63&station=150&station=51&station=6&station=52&station=92&station=112&station=131&station=123&station=95&station=53&station=203&station=190&station=57&station=149&station=148&station=202&station=110&variable=ddtpetp&year=2024&ttype=daily&quick_pick=&begin_date=2024-03-02&end_date=2024-03-02"
df = pd.read_csv(ndawn_url, skiprows=[0, 1, 2, 4])

# Assuming df is your DataFrame
columns_to_print = ['Station Name', 'Latitude', 'Longitude', 'Elevation', 'Year', 'Month', 'Day', 'Penman PET']
print(df[columns_to_print])

    Station Name  Latitude  Longitude  Elevation  Year  Month  Day  Penman PET
0            Ada  47.32119  -96.51406        910  2024      3    2       0.052
1          Adams  48.49988  -98.07588       1580  2024      3    2       0.035
2          Alamo  48.54652 -103.47186       2157  2024      3    2       0.038
3      Alexander  47.75056 -103.73358       2202  2024      3    2       0.019
4       Alvarado  48.24594  -97.02153        809  2024      3    2       0.052
..           ...       ...        ...        ...   ...    ...  ...         ...
188       Wishek  46.26033  -99.66585       2219  2024      3    2       0.050
189      Wolford  48.51603  -99.62441       1610  2024      3    2       0.044
190    Wolverton  46.56545  -96.68726        937  2024      3    2       0.085
191    Ypsilanti  46.77638  -98.52323       1484  2024      3    2       0.045
192      Zeeland  46.01351  -99.68768       2070  2024      3    2       0.058

[193 rows x 8 columns]


# QAQC

In [21]:
#Filter the dataframe to include Minnesota Locations only
# Assuming 'df' is your DataFrame and it contains 'Latitude' and 'Longitude' columns
minnesota_df = df[(df['Latitude'] <= north) & (df['Latitude'] >= south) & (df['Longitude'] <= east) & (df['Longitude'] >= west)]

# Now, printing the specified columns
print(minnesota_df[['Station Name', 'Latitude', 'Longitude', 'Elevation', 'Year', 'Month', 'Day', 'Penman PET']])


        Station Name  Latitude  Longitude  ...  Month  Day  Penman PET
0                Ada  47.32119  -96.51406  ...      3    2       0.052
4           Alvarado  48.24594  -97.02153  ...      3    2       0.052
8                Ayr  47.04639  -97.49481  ...      3    2       0.026
13            Becker  45.34399  -93.85014  ...      3    2       0.091
23          Campbell  46.06494  -96.37014  ...      3    2       0.085
27         Casselton  46.87995  -97.22302  ...      3    2       0.043
31          Clarissa  46.11155  -94.90583  ...      3    2       0.069
51              Ekre  46.54005  -97.14094  ...      3    2       0.061
52        Elbow Lake  45.96308  -95.98682  ...      3    2       0.091
53            Eldred  47.68769  -96.82221  ...      3    2       0.037
54           Emerado  47.91201  -97.32515  ...      3    2       0.041
57             Fargo  46.89693  -96.81221  ...      3    2       0.068
65               Fox  48.87775  -95.85017  ...      3    2       0.046
66    

In [22]:
#Drop the rows that contain outliers above 1 standard deviation above the mean
numMean = minnesota_df["Penman PET"].mean()
numSTD = minnesota_df["Penman PET"].std()

mn_df = minnesota_df.loc[minnesota_df["Penman PET"].between(numMean - numSTD, numMean + numSTD)]
mn_df

,Station Name,Latitude,Longitude,Elevation,Year,Month,Day,Penman PET,Penman PET Flag
0,Ada,47.32119,-96.51406,910,2024,3,2,0.052,NaN
4,Alvarado,48.24594,-97.02153,809,2024,3,2,0.052,NaN
23,Campbell,46.06494,-96.37014,987,2024,3,2,0.085,NaN
27,Casselton,46.87995,-97.22302,938,2024,3,2,0.043,NaN
31,Clarissa,46.11155,-94.90583,1304,2024,3,2,0.069,NaN
51,Ekre,46.54005,-97.14094,1053,2024,3,2,0.061,NaN
54,Emerado,47.91201,-97.32515,877,2024,3,2,0.041,NaN
57,Fargo,46.89693,-96.81221,902,2024,3,2,0.068,NaN
65,Fox,48.87775,-95.85017,1040,2024,3,2,0.046,NaN
66,Foxhome,46.26819,-96.21563,1096,2024,3,2,0.084,NaN


In [25]:
#CSV to feature class conversion
# convert from dataframe to spatially enabled dataframe
local_gdb = r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Lab 2\Lab 2.gdb"
ndawn_spatialdf = arcgis.GeoAccessor.from_xy(mn_df, "Longitude", "Latitude")
ndawn_spatialdf.spatial.to_featureclass(location=os.path.join(local_gdb,"Penman_PET"))

'C:\\Users\\samik\\OneDrive\\Documents\\ArcGIS\\Projects\\Lab 2\\Lab 2.gdb\\Penman_PET'

In [26]:
## GDD
ndawn_GDD_url = r"https://ndawn.ndsu.nodak.edu/table.csv?ttype=cogdd&station=78&station=111&station=98&station=162&station=174&station=142&station=164&station=138&station=161&station=9&station=160&station=159&station=10&station=118&station=56&station=165&station=11&station=12&station=58&station=13&station=84&station=55&station=179&station=7&station=186&station=87&station=14&station=15&station=96&station=191&station=16&station=201&station=137&station=124&station=143&station=17&station=85&station=140&station=134&station=18&station=136&station=65&station=104&station=99&station=192&station=19&station=129&station=20&station=101&station=166&station=178&station=81&station=21&station=97&station=22&station=75&station=184&station=2&station=172&station=139&station=158&station=23&station=157&station=62&station=86&station=24&station=89&station=126&station=167&station=93&station=183&station=90&station=25&station=83&station=107&station=156&station=77&station=26&station=155&station=70&station=127&station=144&station=27&station=173&station=132&station=28&station=195&station=185&station=29&station=30&station=154&station=31&station=187&station=102&station=32&station=119&station=4&station=80&station=33&station=59&station=153&station=105&station=82&station=34&station=198&station=72&station=135&station=35&station=76&station=120&station=141&station=109&station=36&station=79&station=193&station=71&station=37&station=38&station=189&station=39&station=130&station=73&station=188&station=40&station=41&station=54&station=69&station=194&station=145&station=113&station=128&station=42&station=43&station=103&station=171&station=116&station=196&station=88&station=114&station=3&station=163&station=200&station=64&station=115&station=168&station=67&station=175&station=146&station=170&station=197&station=44&station=133&station=106&station=100&station=121&station=45&station=46&station=61&station=66&station=181&station=74&station=60&station=199&station=125&station=176&station=177&station=8&station=180&station=204&station=47&station=122&station=108&station=5&station=152&station=48&station=151&station=147&station=68&station=169&station=49&station=50&station=91&station=182&station=117&station=63&station=150&station=51&station=6&station=52&station=92&station=112&station=131&station=123&station=95&station=53&station=203&station=190&station=57&station=149&station=148&station=202&station=110&year=2024&begin_date=2023-05-01&end_date=2023-08-31"
dataframe_GDD = pd.read_csv(ndawn_GDD_url, skiprows=[0, 1, 2, 4])
# Assuming df is DataFrame
columns_to_print = ['Station Name', 'Latitude', 'Longitude', 'Elevation', 'Year', 'Month', 'Day', 'Max Temp', 'Min Temp','Corn Accumulated Growing Degree Days']
print(dataframe_GDD[columns_to_print])

      Station Name  Latitude  ...  Min Temp  Corn Accumulated Growing Degree Days
0              Ada  47.32119  ...    31.338                                   NaN
1              Ada  47.32119  ...    29.997                                   6.0
2              Ada  47.32119  ...    27.745                                  20.0
3              Ada  47.32119  ...    35.170                                  32.0
4              Ada  47.32119  ...    41.013                                  44.0
...            ...       ...  ...       ...                                   ...
21889      Zeeland  46.01351  ...    55.382                                1927.0
21890      Zeeland  46.01351  ...    50.810                                1942.0
21891      Zeeland  46.01351  ...    50.720                                1957.0
21892      Zeeland  46.01351  ...    54.806                                1974.0
21893      Zeeland  46.01351  ...    57.704                                1995.0

[21894 rows x 1

In [27]:
# Correct DataFrame name and syntax for filtering
minnesota_df_gdd = dataframe_GDD[(dataframe_GDD['Latitude'] <= north) & (dataframe_GDD['Latitude'] >= south) & (dataframe_GDD['Longitude'] <= east) & (dataframe_GDD['Longitude'] >= west)]

# Assuming 'columns_to_print' is a list of column names I want to retain
minnesota_df_gdd = minnesota_df_gdd[columns_to_print]


In [28]:
print(minnesota_df_gdd)

      Station Name  Latitude  ...  Min Temp  Corn Accumulated Growing Degree Days
0              Ada  47.32119  ...    31.338                                   NaN
1              Ada  47.32119  ...    29.997                                   6.0
2              Ada  47.32119  ...    27.745                                  20.0
3              Ada  47.32119  ...    35.170                                  32.0
4              Ada  47.32119  ...    41.013                                  44.0
...            ...       ...  ...       ...                                   ...
21766    Wolverton  46.56545  ...    55.976                                2146.0
21767    Wolverton  46.56545  ...    52.016                                2164.0
21768    Wolverton  46.56545  ...    51.566                                2180.0
21769    Wolverton  46.56545  ...    46.114                                2198.0
21770    Wolverton  46.56545  ...    68.396                                2225.0

[6642 rows x 10

In [29]:
## QAQC

# Drop rows where Min Temp is less than -45 degrees F
filtered_df = minnesota_df_gdd[minnesota_df_gdd['Min Temp'] >= -45]

# Drop rows where Max Temp is greater than 110 degrees F
filtered_df = filtered_df[filtered_df['Max Temp'] <= 110]

print(filtered_df)

      Station Name  Latitude  ...  Min Temp  Corn Accumulated Growing Degree Days
0              Ada  47.32119  ...    31.338                                   NaN
1              Ada  47.32119  ...    29.997                                   6.0
2              Ada  47.32119  ...    27.745                                  20.0
3              Ada  47.32119  ...    35.170                                  32.0
4              Ada  47.32119  ...    41.013                                  44.0
...            ...       ...  ...       ...                                   ...
21766    Wolverton  46.56545  ...    55.976                                2146.0
21767    Wolverton  46.56545  ...    52.016                                2164.0
21768    Wolverton  46.56545  ...    51.566                                2180.0
21769    Wolverton  46.56545  ...    46.114                                2198.0
21770    Wolverton  46.56545  ...    68.396                                2225.0

[6642 rows x 10

In [30]:
#CSV to feature class conversion
# convert from dataframe to spatially enabled dataframe
local_gdb = r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Lab 2\Lab 2.gdb"
ndawn_spatialdfgdd = arcgis.GeoAccessor.from_xy(filtered_df, "Longitude", "Latitude")
ndawn_spatialdfgdd.spatial.to_featureclass(location=os.path.join(local_gdb,"Corn_AGDD"))

'C:\\Users\\samik\\OneDrive\\Documents\\ArcGIS\\Projects\\Lab 2\\Lab 2.gdb\\Corn_AGDD'

### Connect to remote PostGIS

### PostGIS NDAWN dataset

In [31]:
# Evapotranspiration data
arcpy.conversion.FeatureClassToGeodatabase(
    Input_Features="Penman_PET",
    Output_Geodatabase=r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Lab 2\PostgreSQL-34-gis5572(postgres).sde"
)

<Result 'C:\\Users\\samik\\OneDrive\\Documents\\ArcGIS\\Projects\\Lab 2\\PostgreSQL-34-gis5572(postgres).sde'>

In [32]:
# GDD
arcpy.conversion.FeatureClassToGeodatabase(
    Input_Features="Corn_AGDD",
    Output_Geodatabase=r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Lab 2\PostgreSQL-34-gis5572(postgres).sde"
)

<Result 'C:\\Users\\samik\\OneDrive\\Documents\\ArcGIS\\Projects\\Lab 2\\PostgreSQL-34-gis5572(postgres).sde'>

### PostGIS Raster datasets DEM
#### Here , I Implemented functions for downsampling, converting to points, and uploading to PostgreSQL via SDE.
#### Provided paths and parameters for adaptability to diverse datasets and environments.

In [3]:
import arcpy

# Step 1: Downsample Raster
def downsample_raster(input_raster, output_raster, cell_size):
    arcpy.Resample_management(input_raster, output_raster, cell_size)

# Step 2: Convert Raster to Points
def raster_to_points(input_raster, output_points):
    arcpy.RasterToPoint_conversion(input_raster, output_points, "VALUE")

# Step 3: Upload Points to SDE
def upload_points_to_sde(input_points, output_sde_connection, output_sde_feature_class):
    arcpy.FeatureClassToFeatureClass_conversion(input_points, output_sde_connection, output_sde_feature_class)

# Paths and parameters
input_raster = r'C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Lab 2\clipped_dem\Clipped_dem'
output_downsampled_raster = r'C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Lab 2\downsampled_raster.tif'
output_points = r'C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Lab 2\points.shp'
output_sde_connection = r'C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Lab 2\PostgreSQL-34-gis5572(postgres).sde'
output_sde_feature_class = 'points_in_sde'

# Step 1: Downsample Raster
downsample_raster(input_raster, output_downsampled_raster, "15000")

# Step 2: Convert Raster to Points
raster_to_points(output_downsampled_raster, output_points)

# Step 3: Upload Points to SDE
upload_points_to_sde(output_points, output_sde_connection, output_sde_feature_class)


### PostGIS raster Dataset MNLC

In [2]:
import arcpy

# Step 1: Downsample Raster
def downsample_raster(input_raster, output_raster, cell_size):
    arcpy.Resample_management(input_raster, output_raster, cell_size)

# Step 2: Convert Raster to Points
def raster_to_points(input_raster, output_points):
    arcpy.RasterToPoint_conversion(input_raster, output_points, "VALUE")

# Step 3: Upload Points to SDE
def upload_points_to_sde(input_points, output_sde_connection, output_sde_feature_class):
    arcpy.FeatureClassToFeatureClass_conversion(input_points, output_sde_connection, output_sde_feature_class)

# Paths and parameters
input_raster = r'C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Lab 2\clipped_mnlc.tif'
output_downsampled_raster = r'C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Lab 2\mnlc_raster.tif'
output_points = r'C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Lab 2\mnlcpoints.shp'
output_sde_connection = r'C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Lab 2\PostgreSQL-34-gis5572(postgres).sde'
output_sde_feature_class = 'points_in_sde'

# Step 1: Downsample Raster
downsample_raster(input_raster, output_downsampled_raster, "15000")

# Step 2: Convert Raster to Points
raster_to_points(output_downsampled_raster, output_points)

# Step 3: Upload Points to SDE
upload_points_to_sde(output_points, output_sde_connection, output_sde_feature_class)


### temperature to postgis

In [13]:
import json
import psycopg2

# Load the filtered temperature data from JSON file
with open(r'C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Lab 2\mn_temperature\filtered_tem_data.json', 'r') as f:
    data = json.load(f)

# Establish connection to PostgreSQL database
conn = psycopg2.connect(
    dbname="gis5572",
    user="postgres",
    password="%",
    host="34.31.152.38",
)

# Create a cursor object
cur = conn.cursor()

# Create a new table in the database
cur.execute("""
    CREATE TABLE IF NOT EXISTS mn_temperature (
        id SERIAL PRIMARY KEY,
        station TEXT,
        date DATE,
        max_tmpf FLOAT,
        min_tmpf FLOAT,
        name TEXT,
        geometry GEOMETRY(POINT, 4326)
    )
""")

# Loop through the data and insert into the newly created table
for item in data:
    # Extract attributes from JSON data
    station = item['station']
    date = item['date']
    max_tmpf = item['max_tmpf']
    min_tmpf = item['min_tmpf']
    name = item['name']
    geometry = f"POINT({item['geometry']['coordinates'][0]} {item['geometry']['coordinates'][1]})"

    # Define the SQL insert statement
    insert_sql = """
    INSERT INTO mn_temperature (station, date, max_tmpf, min_tmpf, name, geometry)
    VALUES (%s, %s, %s, %s, %s, ST_SetSRID(ST_GeomFromText(%s), 4326))
    """
    
    # Execute the insert statement
    cur.execute(insert_sql, (station, date, max_tmpf, min_tmpf, name, geometry))

# Commit changes to the database
conn.commit()

# Close cursor and connection
cur.close()
conn.close()
